In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

# Environment 

This notebook only works for temperature
- Average = tg_mean
- Maximum = tx_mean
- Minimum = tn_mean


In [35]:
#Select Variable 
var = "tx_mean"


# Run all from here ############

In [36]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

In [37]:
variable85="rcp85_"+var+"_annual"
variable45="rcp45_"+var+"_annual"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_tx_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_tx_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_tx_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_tx_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_tx_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_tx_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_tx_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_tx_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_tx_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_tx_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_tx_mean_annual.nc']

# FOR RCP 45

## Subsetting arrays by periods of time and percentiles with Xclim

In [38]:
dsEns45= ens.create_ensemble(files45)
dsEns45

<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tx_mean      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [39]:
# Mapping data - 30 year means
tmp145 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
window = 30
time145 = tmp145.time[0::window]

ds30yavg45 = tmp145.coarsen(time=window).mean()
ds30yavg45['time'] = time145
perc30yavg45 = ens.ensemble_percentiles(ds30yavg45)
perc30yavg45 # entire grid

# Mapping data

<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 4)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1981-01-01 2011-01-01 ... 2071-01-01
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tx_mean_p10  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    tx_mean_p50  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    tx_mean_p90  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>

## Transform into DataFrame

In [40]:
%time df45 = perc30yavg45.drop('realization').to_dataframe().dropna()

df45

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



CPU times: user 1min 31s, sys: 13min 3s, total: 14min 34s
Wall time: 14 s


tx_mean_p10  tx_mean_p50  tx_mean_p90
lat       lon        time                                             
66.623306 -89.045212 1981-01-01   262.737518   263.038483   263.312561
                     2011-01-01   263.783142   264.386505   265.065247
                     2041-01-01   264.740082   265.552704   267.285309
                     2071-01-01   265.130463   266.151184   267.911316
          -88.961884 1981-01-01   262.853027   263.154175   263.427216
...                                      ...          ...          ...
40.041039 -74.129143 2071-01-01   292.627045   293.923859   294.108551
          -74.045807 1981-01-01   290.763702   291.045258   291.103027
                     2011-01-01   291.677429   292.059174   292.577118
                     2041-01-01   292.113770   293.082367   293.394043
                     2071-01-01   292.423950   293.703125   293.888397

[304824 rows x 3 columns]

## Transform in Celsius and round

In [41]:
df45C = df45.copy()
df45C[var+"_p10"] = round(df45C[var+"_p10"] -273.15,2)
df45C[var+"_p50"] = round(df45C[var+"_p50"] -273.15,2)
df45C[var+"_p90"] = round(df45C[var+"_p90"] -273.15,2)

df45C

tx_mean_p10  tx_mean_p50  tx_mean_p90
lat       lon        time                                             
66.623306 -89.045212 1981-01-01       -10.41       -10.11        -9.84
                     2011-01-01        -9.37        -8.76        -8.08
                     2041-01-01        -8.41        -7.60        -5.86
                     2071-01-01        -8.02        -7.00        -5.24
          -88.961884 1981-01-01       -10.30       -10.00        -9.72
...                                      ...          ...          ...
40.041039 -74.129143 2071-01-01        19.48        20.77        20.96
          -74.045807 1981-01-01        17.61        17.90        17.95
                     2011-01-01        18.53        18.91        19.43
                     2041-01-01        18.96        19.93        20.24
                     2071-01-01        19.27        20.55        20.74

[304824 rows x 3 columns]

## Eliminate 2011, Pivot Table and Create new columns 

In [42]:
Region1i45 = df45C.reset_index()
print (Region1i45)
Region1i245 = Region1i45[Region1i45.time.dt.year!= 2011]
print ("2011 eliminated")
df45f = Region1i245.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df45f.columns = [year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df45f

              lat        lon       time  tx_mean_p10  tx_mean_p50  tx_mean_p90
0       66.623306 -89.045212 1981-01-01       -10.41       -10.11        -9.84
1       66.623306 -89.045212 2011-01-01        -9.37        -8.76        -8.08
2       66.623306 -89.045212 2041-01-01        -8.41        -7.60        -5.86
3       66.623306 -89.045212 2071-01-01        -8.02        -7.00        -5.24
4       66.623306 -88.961884 1981-01-01       -10.30       -10.00        -9.72
...           ...        ...        ...          ...          ...          ...
304819  40.041039 -74.129143 2071-01-01        19.48        20.77        20.96
304820  40.041039 -74.045807 1981-01-01        17.61        17.90        17.95
304821  40.041039 -74.045807 2011-01-01        18.53        18.91        19.43
304822  40.041039 -74.045807 2041-01-01        18.96        19.93        20.24
304823  40.041039 -74.045807 2071-01-01        19.27        20.55        20.74

[304824 rows x 6 columns]
2011 eliminated
DF pivote

hist_rcp45_p10  t2050_rcp45_p10  t2080_rcp45_p10  \
lat       lon                                                            
40.041039 -89.045212           17.32            18.37            18.84   
          -88.961884           17.31            18.36            18.83   
          -88.878548           17.30            18.35            18.82   
          -88.795219           17.22            18.27            18.74   
          -88.711891           17.24            18.29            18.76   
...                              ...              ...              ...   
66.623306 -61.629639           -6.52            -5.00            -5.20   
          -61.546310           -6.88            -5.38            -5.57   
          -61.462978           -6.62            -5.12            -5.31   
          -61.379650           -6.52            -5.03            -5.22   
          -61.296322           -5.80            -4.31            -4.50   

                      hist_rcp45_p50  t2050_rcp45_p50  t2080_rcp45_p50  \
lat       lon                                                            
40.041039 -89.045212           17.51            19.68            20.28   
          -88.961884           17.50            19.67            20.27   
          -88.878548           17.48            19.65            20.26   
          -88.795219           17.40            19.57            20.18   
          -88.711891           17.42            19.59            20.21   
...                              ...              ...              ...   
66.623306 -61.629639           -6.20            -4.07            -3.53   
          -61.546310           -6.56            -4.42            -3.91   
          -61.462978           -6.29            -4.15            -3.66   
          -61.379650           -6.20            -4.05            -3.58   
          -61.296322           -5.49            -3.33            -2.88   

                      hist_rcp45_p90  t2050_rcp45_p90  t2080_rcp45_p90  
lat       lon                                                           
40.041039 -89.045212           17.57            20.23            20.84  
          -88.961884           17.56            20.22            20.82  
          -88.878548           17.54            20.20            20.80  
          -88.795219           17.47            20.12            20.72  
          -88.711891           17.49            20.14            20.74  
...                              ...              ...              ...  
66.623306 -61.629639           -5.94            -3.39            -2.95  
          -61.546310           -6.30            -3.76            -3.30  
          -61.462978           -6.04            -3.56            -3.03  
          -61.379650           -5.95            -3.54            -2.94  
          -61.296322           -5.25            -2.87            -2.21  

[76206 rows x 9 columns]

# FOR RCP 85

## Subsetting arrays by periods of time and percentiles with Xclim

In [43]:
dsEns85= ens.create_ensemble(files85)
dsEns85

<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tx_mean      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [44]:
# Mapping data - 30 year means
tmp185 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
window = 30
time185 = tmp185.time[0::window]

ds30yavg85 = tmp185.coarsen(time=window).mean()
ds30yavg85['time'] = time185
perc30yavg85 = ens.ensemble_percentiles(ds30yavg85)
perc30yavg85 # entire grid

<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 4)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1981-01-01 2011-01-01 ... 2071-01-01
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tx_mean_p10  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    tx_mean_p50  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    tx_mean_p90  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>

## Transform into DataFrame

In [45]:
%time df85 = perc30yavg85.drop('realization').to_dataframe().dropna()
df85

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



CPU times: user 1min 24s, sys: 12min 12s, total: 13min 36s
Wall time: 12.5 s


tx_mean_p10  tx_mean_p50  tx_mean_p90
lat       lon        time                                             
66.623306 -89.045212 1981-01-01   262.806366   262.971771   263.313293
                     2011-01-01   263.916260   264.352875   265.327423
                     2041-01-01   265.882050   266.743713   268.817230
                     2071-01-01   267.450256   269.963348   272.103180
          -88.961884 1981-01-01   262.921112   263.086517   263.428009
...                                      ...          ...          ...
40.041039 -74.129143 2071-01-01   294.526489   296.095184   296.349487
          -74.045807 1981-01-01   290.781067   291.041962   291.126740
                     2011-01-01   291.863708   292.221710   292.518219
                     2041-01-01   292.927429   293.937531   294.029388
                     2071-01-01   294.318726   295.873474   296.130676

[304824 rows x 3 columns]

## Transform in Celsius and round

In [46]:
df85C = df85.copy()
df85C[var+"_p10"] = round(df85C[var+"_p10"] -273.15,2)
df85C[var+"_p50"] = round(df85C[var+"_p50"] -273.15,2)
df85C[var+"_p90"] = round(df85C[var+"_p90"] -273.15,2)
df85C

tx_mean_p10  tx_mean_p50  tx_mean_p90
lat       lon        time                                             
66.623306 -89.045212 1981-01-01       -10.34       -10.18        -9.84
                     2011-01-01        -9.23        -8.80        -7.82
                     2041-01-01        -7.27        -6.41        -4.33
                     2071-01-01        -5.70        -3.19        -1.05
          -88.961884 1981-01-01       -10.23       -10.06        -9.72
...                                      ...          ...          ...
40.041039 -74.129143 2071-01-01        21.38        22.95        23.20
          -74.045807 1981-01-01        17.63        17.89        17.98
                     2011-01-01        18.71        19.07        19.37
                     2041-01-01        19.78        20.79        20.88
                     2071-01-01        21.17        22.72        22.98

[304824 rows x 3 columns]

## Eliminate 2011, Pivot Table and Create new columns 

In [47]:
Region1i85 = df85C.reset_index()
print (Region1i85)
Region1i285 = Region1i85[Region1i85.time.dt.year!= 2011]
print ("2011 eliminated")
df85f = Region1i285.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df85f.columns = [year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df85f

              lat        lon       time  tx_mean_p10  tx_mean_p50  tx_mean_p90
0       66.623306 -89.045212 1981-01-01       -10.34       -10.18        -9.84
1       66.623306 -89.045212 2011-01-01        -9.23        -8.80        -7.82
2       66.623306 -89.045212 2041-01-01        -7.27        -6.41        -4.33
3       66.623306 -89.045212 2071-01-01        -5.70        -3.19        -1.05
4       66.623306 -88.961884 1981-01-01       -10.23       -10.06        -9.72
...           ...        ...        ...          ...          ...          ...
304819  40.041039 -74.129143 2071-01-01        21.38        22.95        23.20
304820  40.041039 -74.045807 1981-01-01        17.63        17.89        17.98
304821  40.041039 -74.045807 2011-01-01        18.71        19.07        19.37
304822  40.041039 -74.045807 2041-01-01        19.78        20.79        20.88
304823  40.041039 -74.045807 2071-01-01        21.17        22.72        22.98

[304824 rows x 6 columns]
2011 eliminated
DF pivote

hist_rcp85_p10  t2050_rcp85_p10  t2080_rcp85_p10  \
lat       lon                                                            
40.041039 -89.045212           17.33            19.36            20.99   
          -88.961884           17.32            19.35            20.98   
          -88.878548           17.31            19.34            20.97   
          -88.795219           17.23            19.26            20.89   
          -88.711891           17.25            19.28            20.92   
...                              ...              ...              ...   
66.623306 -61.629639           -6.34            -4.21            -3.43   
          -61.546310           -6.71            -4.59            -3.81   
          -61.462978           -6.47            -4.33            -3.55   
          -61.379650           -6.40            -4.24            -3.46   
          -61.296322           -5.70            -3.52            -2.74   

                      hist_rcp85_p50  t2050_rcp85_p50  t2080_rcp85_p50  \
lat       lon                                                            
40.041039 -89.045212           17.52            20.61            22.47   
          -88.961884           17.51            20.60            22.47   
          -88.878548           17.49            20.58            22.45   
          -88.795219           17.41            20.50            22.38   
          -88.711891           17.43            20.52            22.40   
...                              ...              ...              ...   
66.623306 -61.629639           -6.13            -2.85            -0.67   
          -61.546310           -6.49            -3.24            -1.04   
          -61.462978           -6.23            -2.99            -0.79   
          -61.379650           -6.14            -2.91            -0.71   
          -61.296322           -5.43            -2.21             0.00   

                      hist_rcp85_p90  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                           
40.041039 -89.045212           17.60            21.00            23.19  
          -88.961884           17.59            20.98            23.18  
          -88.878548           17.58            20.97            23.16  
          -88.795219           17.50            20.89            23.07  
          -88.711891           17.52            20.91            23.09  
...                              ...              ...              ...  
66.623306 -61.629639           -5.91            -2.34             0.31  
          -61.546310           -6.27            -2.71            -0.15  
          -61.462978           -6.01            -2.46             0.03  
          -61.379650           -5.92            -2.41             0.04  
          -61.296322           -5.21            -1.75             0.69  

[76206 rows x 9 columns]

# Merge df45 and 85

In [48]:
dftg = pd.merge(df45f, df85f, on=["lat","lon"])
dftg.head()

hist_rcp45_p10  t2050_rcp45_p10  t2080_rcp45_p10  \
lat       lon                                                            
40.041039 -89.045212           17.32            18.37            18.84   
          -88.961884           17.31            18.36            18.83   
          -88.878548           17.30            18.35            18.82   
          -88.795219           17.22            18.27            18.74   
          -88.711891           17.24            18.29            18.76   

                      hist_rcp45_p50  t2050_rcp45_p50  t2080_rcp45_p50  \
lat       lon                                                            
40.041039 -89.045212           17.51            19.68            20.28   
          -88.961884           17.50            19.67            20.27   
          -88.878548           17.48            19.65            20.26   
          -88.795219           17.40            19.57            20.18   
          -88.711891           17.42            19.59            20.21   

                      hist_rcp45_p90  t2050_rcp45_p90  t2080_rcp45_p90  \
lat       lon                                                            
40.041039 -89.045212           17.57            20.23            20.84   
          -88.961884           17.56            20.22            20.82   
          -88.878548           17.54            20.20            20.80   
          -88.795219           17.47            20.12            20.72   
          -88.711891           17.49            20.14            20.74   

                      hist_rcp85_p10  t2050_rcp85_p10  t2080_rcp85_p10  \
lat       lon                                                            
40.041039 -89.045212           17.33            19.36            20.99   
          -88.961884           17.32            19.35            20.98   
          -88.878548           17.31            19.34            20.97   
          -88.795219           17.23            19.26            20.89   
          -88.711891           17.25            19.28            20.92   

                      hist_rcp85_p50  t2050_rcp85_p50  t2080_rcp85_p50  \
lat       lon                                                            
40.041039 -89.045212           17.52            20.61            22.47   
          -88.961884           17.51            20.60            22.47   
          -88.878548           17.49            20.58            22.45   
          -88.795219           17.41            20.50            22.38   
          -88.711891           17.43            20.52            22.40   

                      hist_rcp85_p90  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                           
40.041039 -89.045212           17.60            21.00            23.19  
          -88.961884           17.59            20.98            23.18  
          -88.878548           17.58            20.97            23.16  
          -88.795219           17.50            20.89            23.07  
          -88.711891           17.52            20.91            23.09

# Extract hist

In [49]:
dftg1 = dftg.drop(columns=["hist_rcp45_p10","hist_rcp45_p50", "hist_rcp45_p90", "hist_rcp85_p10", "hist_rcp85_p50", "hist_rcp85_p90"])
dftgh = dftg[["hist_rcp45_p50", "hist_rcp85_p50"]]

In [50]:
dftg1

t2050_rcp45_p10  t2080_rcp45_p10  t2050_rcp45_p50  \
lat       lon                                                             
40.041039 -89.045212            18.37            18.84            19.68   
          -88.961884            18.36            18.83            19.67   
          -88.878548            18.35            18.82            19.65   
          -88.795219            18.27            18.74            19.57   
          -88.711891            18.29            18.76            19.59   
...                               ...              ...              ...   
66.623306 -61.629639            -5.00            -5.20            -4.07   
          -61.546310            -5.38            -5.57            -4.42   
          -61.462978            -5.12            -5.31            -4.15   
          -61.379650            -5.03            -5.22            -4.05   
          -61.296322            -4.31            -4.50            -3.33   

                      t2080_rcp45_p50  t2050_rcp45_p90  t2080_rcp45_p90  \
lat       lon                                                             
40.041039 -89.045212            20.28            20.23            20.84   
          -88.961884            20.27            20.22            20.82   
          -88.878548            20.26            20.20            20.80   
          -88.795219            20.18            20.12            20.72   
          -88.711891            20.21            20.14            20.74   
...                               ...              ...              ...   
66.623306 -61.629639            -3.53            -3.39            -2.95   
          -61.546310            -3.91            -3.76            -3.30   
          -61.462978            -3.66            -3.56            -3.03   
          -61.379650            -3.58            -3.54            -2.94   
          -61.296322            -2.88            -2.87            -2.21   

                      t2050_rcp85_p10  t2080_rcp85_p10  t2050_rcp85_p50  \
lat       lon                                                             
40.041039 -89.045212            19.36            20.99            20.61   
          -88.961884            19.35            20.98            20.60   
          -88.878548            19.34            20.97            20.58   
          -88.795219            19.26            20.89            20.50   
          -88.711891            19.28            20.92            20.52   
...                               ...              ...              ...   
66.623306 -61.629639            -4.21            -3.43            -2.85   
          -61.546310            -4.59            -3.81            -3.24   
          -61.462978            -4.33            -3.55            -2.99   
          -61.379650            -4.24            -3.46            -2.91   
          -61.296322            -3.52            -2.74            -2.21   

                      t2080_rcp85_p50  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                            
40.041039 -89.045212            22.47            21.00            23.19  
          -88.961884            22.47            20.98            23.18  
          -88.878548            22.45            20.97            23.16  
          -88.795219            22.38            20.89            23.07  
          -88.711891            22.40            20.91            23.09  
...                               ...              ...              ...  
66.623306 -61.629639            -0.67            -2.34             0.31  
          -61.546310            -1.04            -2.71            -0.15  
          -61.462978            -0.79            -2.46             0.03  
          -61.379650            -0.71            -2.41             0.04  
          -61.296322             0.00            -1.75             0.69  

[76206 rows x 12 columns]

# Merge both hist_50 and get the mean

In [51]:
dftgh["hist_p50"] = round((dftgh["hist_rcp45_p50"]+ dftgh["hist_rcp85_p50"])/2, 2)
dftgh2 = dftgh.reset_index()

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftgh["hist_p50"] = round((dftgh["hist_rcp45_p50"]+ dftgh["hist_rcp85_p50"])/2, 2)



In [52]:
dftgh3 = dftgh2[["lat", "lon", "hist_p50"]]
dftgh3

,lat,lon,hist_p50
0,40.041039,-89.045212,17.52
1,40.041039,-88.961884,17.51
2,40.041039,-88.878548,17.48
3,40.041039,-88.795219,17.40
4,40.041039,-88.711891,17.42
...,...,...,...
76201,66.623306,-61.629639,-6.16
76202,66.623306,-61.546310,-6.52
76203,66.623306,-61.462978,-6.26
76204,66.623306,-61.379650,-6.17


# Merge means with the 45 and 85 DF

In [53]:
dftg_all = pd.merge(dftgh3, dftg1, on=["lat","lon"])

In [54]:
dftg_all

,lat,lon,hist_p50,t2050_rcp45_p10,t2080_rcp45_p10,t2050_rcp45_p50,t2080_rcp45_p50,t2050_rcp45_p90,t2080_rcp45_p90,t2050_rcp85_p10,t2080_rcp85_p10,t2050_rcp85_p50,t2080_rcp85_p50,t2050_rcp85_p90,t2080_rcp85_p90
0,40.041039,-89.045212,17.52,18.37,18.84,19.68,20.28,20.23,20.84,19.36,20.99,20.61,22.47,21.00,23.19
1,40.041039,-88.961884,17.51,18.36,18.83,19.67,20.27,20.22,20.82,19.35,20.98,20.60,22.47,20.98,23.18
2,40.041039,-88.878548,17.48,18.35,18.82,19.65,20.26,20.20,20.80,19.34,20.97,20.58,22.45,20.97,23.16
3,40.041039,-88.795219,17.40,18.27,18.74,19.57,20.18,20.12,20.72,19.26,20.89,20.50,22.38,20.89,23.07
4,40.041039,-88.711891,17.42,18.29,18.76,19.59,20.21,20.14,20.74,19.28,20.92,20.52,22.40,20.91,23.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-6.16,-5.00,-5.20,-4.07,-3.53,-3.39,-2.95,-4.21,-3.43,-2.85,-0.67,-2.34,0.31
76202,66.623306,-61.546310,-6.52,-5.38,-5.57,-4.42,-3.91,-3.76,-3.30,-4.59,-3.81,-3.24,-1.04,-2.71,-0.15
76203,66.623306,-61.462978,-6.26,-5.12,-5.31,-4.15,-3.66,-3.56,-3.03,-4.33,-3.55,-2.99,-0.79,-2.46,0.03
76204,66.623306,-61.379650,-6.17,-5.03,-5.22,-4.05,-3.58,-3.54,-2.94,-4.24,-3.46,-2.91,-0.71,-2.41,0.04


In [55]:
dftg_all.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_annual.feather")

In [56]:
#dftg_all = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_annual.feather")

In [57]:
dftg_all 

,lat,lon,hist_p50,t2050_rcp45_p10,t2080_rcp45_p10,t2050_rcp45_p50,t2080_rcp45_p50,t2050_rcp45_p90,t2080_rcp45_p90,t2050_rcp85_p10,t2080_rcp85_p10,t2050_rcp85_p50,t2080_rcp85_p50,t2050_rcp85_p90,t2080_rcp85_p90
0,40.041039,-89.045212,17.52,18.37,18.84,19.68,20.28,20.23,20.84,19.36,20.99,20.61,22.47,21.00,23.19
1,40.041039,-88.961884,17.51,18.36,18.83,19.67,20.27,20.22,20.82,19.35,20.98,20.60,22.47,20.98,23.18
2,40.041039,-88.878548,17.48,18.35,18.82,19.65,20.26,20.20,20.80,19.34,20.97,20.58,22.45,20.97,23.16
3,40.041039,-88.795219,17.40,18.27,18.74,19.57,20.18,20.12,20.72,19.26,20.89,20.50,22.38,20.89,23.07
4,40.041039,-88.711891,17.42,18.29,18.76,19.59,20.21,20.14,20.74,19.28,20.92,20.52,22.40,20.91,23.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-6.16,-5.00,-5.20,-4.07,-3.53,-3.39,-2.95,-4.21,-3.43,-2.85,-0.67,-2.34,0.31
76202,66.623306,-61.546310,-6.52,-5.38,-5.57,-4.42,-3.91,-3.76,-3.30,-4.59,-3.81,-3.24,-1.04,-2.71,-0.15
76203,66.623306,-61.462978,-6.26,-5.12,-5.31,-4.15,-3.66,-3.56,-3.03,-4.33,-3.55,-2.99,-0.79,-2.46,0.03
76204,66.623306,-61.379650,-6.17,-5.03,-5.22,-4.05,-3.58,-3.54,-2.94,-4.24,-3.46,-2.91,-0.71,-2.41,0.04


# Merge complete DF with Polygons for each spatial scale

## Dictionary - regioins: column name, short name

In [58]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
              "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"), 
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [59]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [60]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "à": "a", "è": "e", "ô" : "o", "ç":"c", "É": "E", "È": "E", "Î": "i", "Ç":"C" } 

In [61]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftg_all, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_"+var+"_annual.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_"+var+"_annual.json", driver="GeoJSON")

  warnings.warn(



DDE_STF_20K_REG_FOR_VUE_S
Merged with polygons
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC


  warnings.warn(



DDE_STF_20K_UA_PER_VUE_S
Merged with polygons
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352


  warnings.warn(



...

Massif des monts Groulx


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Coteaux des lacs Matonipi et Jonquet_tx_mean_annual.json: No such file or directory


Coteaux des lacs Matonipi et Jonquet


  warnings.warn(



territoire_guide
Merged with polygons
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
